# Structure 이해

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1, 1, 28, 28)   # batch size, channel, height, width
print(f"tensor size: {inputs.shape}")

tensor size: torch.Size([1, 1, 28, 28])


In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1) # 1channel을 입력 받아 32channel을 뽑아냄.
                                      # kernel size = 3, padding = 1
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
pool = nn.MaxPool2d(2)   # 정수 하나를 인자로 넣으면 kernel size, stride가 둘 다 해당값으로 지정됨
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [6]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [7]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [8]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [9]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


## view()로 tensor를 펼침

In [10]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [11]:
#input_dim = 3136, output_dim = 10
fc = nn.Linear(out.shape[1], 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


# CNN으로 MNIST 분류하기

In [12]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from tqdm import tqdm_notebook

In [13]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# torch.manual_seed(77)

# if device == 'cuda':
#     torch.cuda.manual_seed_all(77)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(77)
if device == 'cuda':
    torch.cuda.manual_seed_all(77)
print(f"uning: {device}")



uning: cuda


In [14]:
learning_rate = 1e-3
training_epochs = 200
batch_size = 100

In [15]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import os

def get_alphabet(root: str, batch_size: int):
    
    train_path = os.path.join(root, 'train')
    test_path = os.path.join(root, 'test')
    
    alphabet_train1 = ImageFolder(root = train_path,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
#                                      transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
                                     transforms.Grayscale(1),
                                     transforms.RandomRotation(5),
                                     transforms.RandomInvert()
                                 ]),
                                 target_transform=None)
    
    alphabet_train2 = ImageFolder(root = train_path,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
#                                     transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
                                     transforms.Grayscale(1)
                                 ]),
                                 target_transform=None)
    
    alphabet_train3 = ImageFolder(root = train_path,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
#                                     transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
                                     transforms.Grayscale(1),
                                     transforms.CenterCrop(20),
                                     transforms.Resize(28)
                                 ]),
                                 target_transform=None)
    
    
    
    alphabet_test = ImageFolder(root = test_path,
                                 transform=transforms.Compose([
                                     transforms.ToTensor(),
#                                      transforms.Normalize(mean=(0.1307,), std=(0.3081,)),
                                     transforms.Grayscale(1)
                                 ]),
                                 target_transform=None)
    
    train_loader = DataLoader(alphabet_train1,
                              batch_size=batch_size,
                              shuffle=True,
                              drop_last=True,
                              num_workers=8)

    test_loader = DataLoader(alphabet_test,
                             batch_size=batch_size,
                             shuffle=False,
                             drop_last=False,
                             num_workers=8) 
    
    return (train_loader, test_loader)

dset_root = './data'
train_loader, test_loader = get_alphabet(root = dset_root, batch_size = batch_size)

In [16]:
# mnist_train = dsets.MNIST(root = 'MNIST_data/',
#                           train=True, # train data로 download
#                           transform=transforms.ToTensor(),
#                           download=True)

# mnist_test = dsets.MNIST(root = 'MNIST_data/',
#                           train=False, # train data로 download
#                           transform=transforms.ToTensor(),
#                           download=True)

# data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
#                                           batch_size=batch_size,
#                                           shuffle=True,
#                                           drop_last=True)


## Class model 설계

In [17]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Layer 1
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # Layer 2 
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7*7*64, 26, bias=True)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # flatten for 전결합층
        out = self.fc(out)
        return out

In [18]:
# class CNN(torch.nn.Module):
#     def __init__(self, input_size, num_classes):
#         super(CNN, self).__init__()
#         # Layer 1
#         self.layer1 = torch.nn.Sequential(
#             torch.nn.Conv2d(input_size[0], 32, kernel_size=5),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(kernel_size=2))
        
#         # Layer 2 
#         self.layer2 = torch.nn.Sequential(
#             torch.nn.Conv2d(32, 64, kernel_size=5),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(kernel_size=2))
        
#         # 전결합층 7x7x64 inputs -> 10 outputs
#         self.fc = torch.nn.Linear(4*4*64, num_classes)
        
#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = out.view(out.size(0), -1) # flatten for 전결합층
#         out = self.fc(out)
#         return out

In [19]:
model = CNN().to(device)

# criterion = torch.nn.CrossEntropyLoss().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

# total_batch = len(data_loader)
# print(f"총 배치의 수: {total_batch}")
# model = CNN((1, 28, 28), 10).to(device)
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
criterion = torch.nn.CrossEntropyLoss().to(device)  # loss function
# train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=64, shuffle=True)
# test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=64, shuffle=True)


In [20]:
# for epoch in range(training_epochs):
#     avg_cost = 0

#     for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
#         # image is already size of (28x28), no reshape
#         # label is not one-hot encoded
#         X = X.to(device)
#         Y = Y.to(device)

#         optimizer.zero_grad()
#         hypothesis = model(X)
#         cost = criterion(hypothesis, Y)
#         cost.backward()
#         optimizer.step()

#         avg_cost += cost / total_batch

#     print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
import numpy as np

for epoch in range(training_epochs):
    train_loss = []
    
    for i, (data, labels) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):
        # pass data through network
        data = data.to(device)
        labels = labels.to(device)
        
        outputs = model(data)
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        
    test_loss = []
    test_accuracy = []
    
    for i, (data, labels) in enumerate(test_loader):
        data = data.to(device)
        labels = labels.to(device)
        
        # pass data through network
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        test_loss.append(loss.item())
        test_accuracy.append((predicted == labels).sum().item() / predicted.size(0))
    print('epoch: {}, train loss: {}, test loss: {}, test accuracy: {}'.format(epoch, np.mean(train_loss), np.mean(test_loss), np.mean(test_accuracy)))

<ipython-input-20-371649f15f24>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (data, labels) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):


  0%|          | 0/256 [00:00<?, ?it/s]

epoch: 0, train loss: 2.278970491141081, test loss: 1.4254045373988602, test accuracy: 0.610048094709582


  0%|          | 0/256 [00:00<?, ?it/s]

epoch: 1, train loss: 1.182452634908259, test loss: 1.0180785869652371, test accuracy: 0.7383388827229004


...

epoch: 123, train loss: 0.02903398894136444, test loss: 0.26386780859375336, test accuracy: 0.9568220495745466


  0%|          | 0/256 [00:00<?, ?it/s]

epoch: 124, train loss: 0.025635577986804492, test loss: 0.23690119696745895, test accuracy: 0.9620902700702925


  0%|          | 0/256 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
#     X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
#     Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

NameError: name 'mnist_test' is not defined